In [32]:
%matplotlib inline
import numpy as np
import matplotlib
import requests
import os
import ntpath
import pandas as pd
matplotlib.rcParams['figure.figsize'] = (16.0,16.0)

In [27]:
URL = 'http://moca.usc.edu:5000'
data = {'genome': 'hg19', 'format':'json'}
request_data = []

In [31]:
for file in os.listdir("/home/saket/Desktop/chip_scoring_beds/"):
    if file.endswith(".quest"):
        filename = "/home/saket/Desktop/chip_scoring_beds/{}".format(file)
        files = {'file': open(filename, 'rb')}
        fn = ntpath.basename(filename)
        req = requests.post(URL, files=files, data=data)
        request_data.append((fn,req.json()['job_id']))

/home/saket/Desktop/chip_scoring_beds/K562_NFRKB_A301-459A_Rep1_Rep2_RepAll_peaks.quest
K562_NFRKB_A301-459A_Rep1_Rep2_RepAll_peaks.quest		64b2a5a0-2b59-4f8b-9f7c-f1d393dd8087

/home/saket/Desktop/chip_scoring_beds/MCF7_GABPalpha_sc-22810_Rep1vsRep2_RepAll_peaks.quest
MCF7_GABPalpha_sc-22810_Rep1vsRep2_RepAll_peaks.quest		5a6136dd-28e1-4af8-87bb-a3c71aa16f61

/home/saket/Desktop/chip_scoring_beds/HepG2_Ski_sc-9140_Rep_2-3_RepAll_peaks.quest
HepG2_Ski_sc-9140_Rep_2-3_RepAll_peaks.quest		51614b4a-0dcb-4047-bee0-7b76b0710654

/home/saket/Desktop/chip_scoring_beds/K562_FOXK2_A301-730A_Rep1-2_RepAll_peaks.quest
K562_FOXK2_A301-730A_Rep1-2_RepAll_peaks.quest		7d73839e-eee5-463d-811e-fbe056fdc53b

/home/saket/Desktop/chip_scoring_beds/K562_IKZF1_GTX129438_Rep1_Rep2_RepAll_peaks.quest
K562_IKZF1_GTX129438_Rep1_Rep2_RepAll_peaks.quest		2cf479b5-5630-48b7-8f59-a9eb0bd27af0

/home/saket/Desktop/chip_scoring_beds/K562_E2F1_3742S_Rep1_Rep2_RepAll_peaks.quest
K562_E2F1_3742S_Rep1_Rep2_RepAll_peaks.q

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
df = pd.DataFrame(columns = ('filename', 'link'))
for x, d in enumerate(request_data):
    df[x] = [d[0] d[1]]

In [34]:
df

<a href=''>Test</a>
